In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import csv
import glob
import os

In [2]:
import win32com.client
instCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")
print(instCpCybos.IsConnect)

1


In [3]:
samsung_stoke_data_base = r'D:\dataset\stoke\samsung\data'
samsung_stoke_label_base = r'D:\dataset\stoke\samsung\label'
oil_data_base = r'D:\dataset\stoke\oil\data'
oil_label_base = r'D:\dataset\stoke\oil\label'
ex_bio_data_base = r'D:\dataset\stoke\ex_bio\data'
ex_bio_label_base = r'D:\dataset\stoke\ex_bio\label'
usecols = ['time', 'price', 'sell', 'buy', 'trading_volum']

## 종목 코드: A를 앞에 붙이면 됨

- ### CpSvrNew7224  투자자(기관,외국인,개인 등등) 들의 일일 매매 동향데이터(매수,매도,순매수)를 요청하고 수신합니다.

- ### 

## DATA를 뽑음

In [4]:
def minute_chart(code, k, m_ToDate, m_FromDate):
    ### k분봉 차트 DF로 저장
    StockChart = win32com.client.Dispatch("CpSysDib.StockChart")
    StockChart.SetInputValue(0, code)
    StockChart.SetInputValue(1, ord('1'))  #개수 입력 옵션
    StockChart.SetInputValue(2, m_ToDate)  #시작일자
    StockChart.SetInputValue(3, m_FromDate)  # 끝일자
    StockChart.SetInputValue(5, (0, 1, 2, 3, 4, 5, 8)) # 날짜(0), 시간(1), 시가(2), 고가(3), 저가(4), 종가(5), 거래량(8)
    StockChart.SetInputValue(6, ord('m')) # 분 차트
    StockChart.SetInputValue(7, k) # k개 캔들 주기로, 즉 k분봉
    StockChart.SetInputValue(9, ord('1')) # 수정주가 적용
    StockChart.BlockRequest()
    num = int(StockChart.GetHeaderValue(3))
    df = DataFrame(np.arange(num), columns = ['index'])
    
    def rdt_CP(i):
        # 인덱스에 해당하는 날짜와 시간을 CybosPlus API에서 읽어들임
        return [StockChart.GetDataValue(0, i), StockChart.GetDataValue(1, i)]
    
    def cdt(date, time):
        # 분리된 날짜와 시간을 datetime.datetime 자료형으로 변환
        date = str(date)
        time = str(time)
        if len(time) == 3:
            time = '0' + time
        DT = date + ' ' + time
        return (DT)
    
    df['when'] = df['index'].apply(lambda i : cdt(rdt_CP(num -1 -i)[0], rdt_CP(num - 1 - i )[1]))
    df['open'] = df['index'].apply(lambda i : StockChart.GetDataValue(2, num - 1 -i))
    df['high'] = df['index'].apply(lambda i : StockChart.GetDataValue(3, num - 1 -i))
    df['low'] = df['index'].apply(lambda i : StockChart.GetDataValue(4, num - 1 -i))
    df['close'] = df['index'].apply(lambda i : StockChart.GetDataValue(5, num - 1 -i))
    df['volume'] = df['index'].apply(lambda i : StockChart.GetDataValue(6, num - 1 -i))
    df.drop(['index'], axis = 1, inplace = True)
    return df


### samsung같은 경우는 종목 코드 앞에 A를 붙이기

In [27]:
datas = minute_chart('Q530031', 1, '20201124', '20201012')

In [28]:
datas

,when,open,high,low,close,volume
0,20201112 1113,340,340,340,340,20
1,20201112 1114,340,340,340,340,327
2,20201112 1115,335,340,335,340,4357
3,20201112 1118,335,335,335,335,9426
4,20201112 1119,340,340,335,340,3386
...,...,...,...,...,...,...
2851,20201124 1517,365,365,365,365,57350
2852,20201124 1518,365,365,360,360,12258
2853,20201124 1519,360,365,360,365,1612
2854,20201124 1520,360,365,360,365,60464


In [29]:
datas.to_csv(r'D:\dataset\stoke\new_total\data' + '\\sam_ETN_oil.csv', index=False)

## label 뽑음

In [10]:
data_path = r'D:\dataset\stoke\new_total\data'
label_path = r'D:\dataset\stoke\new_total\label'

In [30]:
data_csv= open(data_path + '\\sam_ETN_oil.csv')
data = csv.reader(data_csv)
load_data_df = pd.DataFrame(data) 
        
tmp_label = []
for i in load_data_df[1][1:]:
    int_label = float(i)
    tmp_label.append(int_label)
del int_label
        
labels = []
for i in range(1, len(tmp_label)):
    if (tmp_label[i-1] - tmp_label[i]) > 0:
        labels.append(2)
    elif (tmp_label[i-1] - tmp_label[i]) == 0:
        labels.append(0)
    else:
        labels.append(1)
        
labels = np.array(labels)
np.save(label_path + '\\sam_ETN_oilchem.npy', labels)